# Marker investigation

In this notebook the variants in the [Pv4 data release](https://www.malariagen.net/resource/30) are used to identify regions of the core genome that are microhaplotype candidates (<200 bp length). The notebook consists of three parts :
- Loading the data
- Subset the variants and samples
- Perform a sliding window across the core regions of the genome calculating certain statistics for each window

Files used in this notebook are available through the [Pv4 data release](https://www.malariagen.net/resource/30), but are also attached to the repo.

This notebook could be used as a template to further analyse microhaplotype candidates yourself, changing the filters to subset the data and the statistics calculated for each window.

## Setup

In [1]:
#!pip install -q --no-warn-conflicts malariagen-data==7.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.5/302.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.8 MB/s 

In [1]:
from malariagen_data.pv4 import Pv4
import pandas as pd
import numpy as np
import allel
# import dask.array as da
# import collections
# import math

In [2]:
np.__version__

'1.23.5'

In [3]:
# Supress warning
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# Load Data

In this notebook we use the [malariagen_data Python package](https://github.com/malariagen/malariagen-data-python) to access the Pv4 data stored on the cloud. Further information on using this package can be found [here](https://malariagen.github.io/parasite-data/landing-page.html).

We initialise access to the data with the below line of code:  

In [4]:
pv4 = Pv4()

Using this we can load the **sample metadata**

In [5]:
pv4_metadata = pv4.sample_metadata()

pv4_metadata.head()

,Sample,Study,Site,First-level administrative division,Country,Lat,Long,Year,ENA,All samples same individual,Population,% callable,QC pass,Exclusion reason,Is returning traveller
0,BBH-1-125,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678989,BBH-1-125,AF,88.52,True,Analysis_set,False
1,BBH_1_132,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678991,BBH_1_132,AF,90.20,True,Analysis_set,False
2,BBH_1_137,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2679003,BBH_1_137,AF,87.09,True,Analysis_set,False
3,BBH_1_153,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678992,BBH_1_153,AF,90.60,True,Analysis_set,False
4,BBH_1_162,X0009-PV-ET-LO,Jimma,Ethiopia: Oromia,Ethiopia,7.683331,36.851318,2016,ERR2678993,BBH_1_162,AF,91.67,True,Analysis_set,False


In [6]:
variant_dataset = pv4.variant_calls(extended=True)
variant_dataset

<xarray.Dataset>
Dimensions:                                   (variants: 4571056, alleles: 7,
                                               samples: 1895, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(1895,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 64, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(65536, 6), meta=np.ndarray>

# Subset Samples
Now we have access to the metadata and variants we can apply fiters to subset the data.

In our analysis we have only included samples which:
* Are **clonal** (FWS>0.95)
* Have **percent callable > 50%**
* Are **unique** (we identify these using the "Exclusion reason" in_analysis_set in the metadata)
* Due to data restrictions at the time of analysis we only include samples that were in Pv1 or certain studies listed below

A copy of the [FWS values](https://www.malariagen.net/sites/default/files/Pv4_fws.txt) which were calculated for Pv4 are stored within this repository. There is also a file listing if a sample was included in Pv1 (an older public release of vivax data).
Below we load the FWS and Pv1 information and add it to the existing metadata:

In [7]:
pv4_fws = pd.read_csv('../supplementary_files/Pv4_fws.txt', sep='\t', comment='t')
pv4_metadata = pd.merge(pv4_metadata, pv4_fws, on='Sample', how='outer')

In [8]:
pv1_samples = pd.read_csv('../supplementary_files/Samples_in_Pv1.tsv', sep='\t')
pv4_metadata = pv4_metadata.merge(pv1_samples, on='Sample', how='left')

Next we identify samples that meet our criteria and filter the variants to only include these samples.

In [9]:
useable_studies = [
    '1128-PV-MULTI-GSK', '1154-PV-TH-PRICE', '1157-PV-MULTI-PRICE', 'X0001-PV-MULTI-HUPALO2016',
    'X0002-PV-KH-PAROBEK2016'
]

loc_filtered_samples = (
    (pv4_metadata["Study"].isin(useable_studies) | pv4_metadata.in_pv_10)
    & (pv4_metadata["Fws"] > 0.95)
    & (pv4_metadata["% callable"] > 50)
    & (pv4_metadata["Exclusion reason"] == "Analysis_set")
)
subset_metadata = pv4_metadata[loc_filtered_samples]
variant_dataset_filtered_samples = variant_dataset.isel(samples=loc_filtered_samples)
variant_dataset_filtered_samples

<xarray.Dataset>
Dimensions:                                   (variants: 4571056, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(615,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(65536, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(65536,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 31, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(65536, 6), meta=np.ndarray>

After applying these filters there are 615 samples left in the analysis set.

## Subset variants

We also apply filters to the variants to only include ones which **pass filters** in the MalariaGEN dataset and are **coding SNPs**  

In [10]:
filters = (
    (variant_dataset_filtered_samples["variant_filter_pass"].data)
    & (variant_dataset_filtered_samples["variant_is_snp"].data)
    & (variant_dataset_filtered_samples["variant_CDS"].data)
)

In [11]:
%%time
variant_dataset_filtered_variants = variant_dataset_filtered_samples.isel(variants=filters.compute())

CPU times: user 2.43 s, sys: 274 ms, total: 2.7 s
Wall time: 46.7 s


We further filter these to only include **biallelic** snps

In [12]:
biallelic_filter = (variant_dataset_filtered_variants["variant_numalt"] == 1).data
variant_dataset_filtered = variant_dataset_filtered_variants.isel(variants=biallelic_filter.compute())
variant_dataset_filtered

<xarray.Dataset>
Dimensions:                                   (variants: 440222, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(615,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(9339, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(9339,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(9339, 31, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(9339,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(9339, 6), meta=np.ndarray>

Now we filter these variants to only include ones which have **high global minor allele frequencies (> 0.1) and low missingness (< 0.1)**.

To do this we must first perform an allele count on the genotypes and convert to frequency

In [13]:
%%time
# allele frequency for all samples
gt = allel.GenotypeDaskArray(variant_dataset_filtered["call_genotype"].data)
ac_pop = gt.count_alleles()
ac_pop_freq = ac_pop.to_frequencies().compute()

CPU times: user 1min 22s, sys: 5.35 s, total: 1min 27s
Wall time: 6min 54s


In [14]:
np.save('ac_pop_freq.npy', ac_pop_freq)

In [13]:
# If the above has already been saved we can load
ac_pop_freq = np.load('ac_pop_freq.npy')

Below we calculate the missingness frequency for each SNP

In [16]:
%%time
freq_missing = gt.count_missing(axis=1).compute() / gt.shape[1]

CPU times: user 44.5 s, sys: 2.76 s, total: 47.2 s
Wall time: 2min 53s


In [17]:
np.save('freq_missing.npy', freq_missing)

In [14]:
# if above has already be run can just load from file
freq_missing = np.load('freq_missing.npy')

We can now use the allele and missingness frequencies to filter the variants to only include minor allele frequency over 0.1 and missingness less than 0.1

In [15]:
pop_freq_filter = (ac_pop_freq.min(axis=1) > 0.1) & (freq_missing < 0.1)
variant_dataset_filtered = variant_dataset_filtered.isel(variants=pop_freq_filter)
variant_dataset_filtered

<xarray.Dataset>
Dimensions:                                   (variants: 13084, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_chrom                             (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    sample_id                                 (samples) object dask.array<chunksize=(615,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(247, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) bool dask.array<chunksize=(247,), meta=np.ndarray>
    variant_is_snp                            (variants) bool dask.array<chunksize=(247,), meta=np.ndarray>
    variant_numalt                            (variants) int32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_CDS                               (variants) bool dask.array<chunksize=(247,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) int8 dask.array<chunksize=(247, 31, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(247,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(247,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) int32 dask.array<chunksize=(247, 6), meta=np.ndarray>

In [32]:
vvv = variant_dataset_filtered.where( ((variant_dataset_filtered['variant_chrom'] == 'PvP01_05_v1') &
                                      (variant_dataset_filtered['variant_position'] >= 1017424) &
                                      (variant_dataset_filtered['variant_position'] <= 1017623)).compute(), drop=True)

In [33]:
vvv

<xarray.Dataset>
Dimensions:                                   (variants: 1, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 1017604
    variant_chrom                             (variants) object 'PvP01_05_v1'
    sample_id                                 (samples) object dask.array<chunksize=(615,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object dask.array<chunksize=(1, 1), meta=np.ndarray>
    variant_filter_pass                       (variants) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    variant_is_snp                            (variants) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    variant_numalt                            (variants) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    variant_CDS                               (variants) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    call_genotype                             (variants, samples, ploidy) float32 dask.array<chunksize=(1, 31, 2), meta=np.ndarray>
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object dask.array<chunksize=(1,), meta=np.ndarray>
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object dask.array<chunksize=(1,), meta=np.ndarray>
    variant_SOR                               (variants) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    variant_VQSLOD                            (variants) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    variant_VariantType                       (variants) object dask.array<chunksize=(1,), meta=np.ndarray>
    variant_altlen                            (variants, alt_alleles) float64 dask.array<chunksize=(1, 6), meta=np.ndarray>

In [38]:
vvv.compute()

<xarray.Dataset>
Dimensions:                                   (variants: 1, alleles: 7,
                                               samples: 615, ploidy: 2,
                                               genotypes: 3, alt_alleles: 6)
Coordinates:
    variant_position                          (variants) int32 1017604
    variant_chrom                             (variants) object 'PvP01_05_v1'
    sample_id                                 (samples) object 'Brazil32' ......
Dimensions without coordinates: variants, alleles, samples, ploidy, genotypes,
                                alt_alleles
Data variables: (12/42)
    variant_allele                            (variants, alleles) object 'C' ...
    variant_filter_pass                       (variants) float64 1.0
    variant_is_snp                            (variants) float64 1.0
    variant_numalt                            (variants) float64 1.0
    variant_CDS                               (variants) float64 1.0
    call_genotype                             (variants, samples, ploidy) float32 ...
    ...                                        ...
    variant_SNPEFF_IMPACT                     (variants) object 'MODERATE'
    variant_SNPEFF_TRANSCRIPT_ID              (variants) object 'PVP01_052510...
    variant_SOR                               (variants) float32 0.657
    variant_VQSLOD                            (variants) float32 0.055
    variant_VariantType                       (variants) object 'SNP'
    variant_altlen                            (variants, alt_alleles) float64 ...

## Load core region data

When we start calculating statistics on microhaplotype candidates we want to make sure to restrict the regions we are looking at to the core genome. This information is stored in [Pv4 regions](https://www.malariagen.net/sites/default/files/Pv4_regions.bed.gz), which we load into a pandas dataframe. This file details the chromosome, the start and end, and the type of the region.

In [20]:
pv4_regions = pd.read_csv(
    "../supplementary_files/Pv4_regions.bed", sep="\t", comment="t", names=["chrom", "chromStart", "chromEnd", "name"]
)
pv4_regions.head()

,chrom,chromStart,chromEnd,name
0,PvP01_01_v1,0,116541,Sub
1,PvP01_01_v1,116541,677962,Core
2,PvP01_01_v1,677962,679789,Cen
3,PvP01_01_v1,679789,903591,Core
4,PvP01_01_v1,903591,1021664,Sub



This file is 0-based, so we convert it to be 1-based below so it is consistent with our variant data

In [21]:
pv4_regions[["chromStart", "chromEnd"]] += 1

Below we count the number of variants in our filtered list that are within the core regions to get a total number of variants that will be included in our analysis

In [22]:
# We probs don't need this

total_variants = 0

for index, row in (pv4_regions.loc[pv4_regions.name == "Core"]).iterrows():
    variant_dataset_chrom = variant_dataset_filtered.where((variant_dataset_filtered.variant_chrom == row.chrom).compute(), drop=True)

    test_variants = variant_dataset_chrom.set_index(
        variants="variant_position", samples="sample_id"
    )

    variant_count = test_variants.sel(
        variants=slice(row.chromStart, row.chromEnd)
    ).dims["variants"]

    print(row.chrom, ':', row.chromStart, '-', row.chromEnd, ':', variant_count)

    total_variants += variant_count
print("Total variants to be included in analysis : ", total_variants)

PvP01_01_v1 : 116542 - 677963 : 369
PvP01_01_v1 : 679790 - 903592 : 157
PvP01_02_v1 : 100156 - 162349 : 30
PvP01_02_v1 : 164088 - 745644 : 415
PvP01_03_v1 : 108062 - 630664 : 582
PvP01_03_v1 : 632482 - 894723 : 209
PvP01_04_v1 : 185115 - 564966 : 268
PvP01_04_v1 : 566928 - 685686 : 81
PvP01_04_v1 : 748924 - 967651 : 205
PvP01_05_v1 : 143102 - 844199 : 420
PvP01_05_v1 : 846073 - 1408237 : 317
PvP01_06_v1 : 39349 - 335698 : 231
PvP01_06_v1 : 337570 - 1017210 : 481
PvP01_07_v1 : 60454 - 1256419 : 821
PvP01_07_v1 : 1258300 - 1463495 : 177
PvP01_08_v1 : 28424 - 1132375 : 595
PvP01_08_v1 : 1134203 - 1627673 : 298
PvP01_09_v1 : 255762 - 977412 : 420
PvP01_09_v1 : 979223 - 2156012 : 675
PvP01_10_v1 : 140990 - 1011339 : 584
PvP01_10_v1 : 1013168 - 1327962 : 252
PvP01_10_v1 : 1375230 - 1472801 : 97
PvP01_11_v1 : 62172 - 1603557 : 799
PvP01_11_v1 : 1605512 - 2051115 : 272
PvP01_12_v1 : 54909 - 792293 : 364
PvP01_12_v1 : 818497 - 1048750 : 146
PvP01_12_v1 : 1050262 - 3028219 : 1027
PvP01_13_v1 : 3

# Sliding window through core regions

The co genome was then scanned in coding regions for all 200 bp windows in which > 1 of the identified variants were found and filtered for high diversity (global heterozygosity ≥0.5).

The function `evaluate_marker_options` below utilises the other functions definded to perform a sliding window through the core regions of the definded chromosome. For each window the following is calculated:
- the number of biallelic snps in the window
- for each unique allele how many samples have that allele

In [23]:
def filter_variants(variant_dataset, field, value):
    filter_values = (variant_dataset[field] == value).data.compute()
    variant_dataset_filtered = variant_dataset.isel(variants=filter_values)
    return variant_dataset_filtered


def variant_positions(positions):
    return list(positions)


def unique_allele_counts_in_window(gt):
    unique, index, counts = np.unique(gt, axis=1, return_counts=True, return_index=True)
    # Find index with the missing or het
    alleles_with_missing = []
    alleles_with_het = []
    for i in range(len(index)):
        if -1 in (gt[:, index[i]].compute()):
            alleles_with_missing.append(i)
        if True in gt[:, int(index[i])].is_het().compute():
            alleles_with_het.append(i)

    return counts, alleles_with_missing, alleles_with_het


def calculate_stats(variant_dataset, window_length, step):
    pos = variant_dataset["variants"].data

    # Count variants in each window
    n_variants, windows = allel.windowed_count(
        pos, size=window_length, step=step
    )

    # Exclude windows with no variants
    index_with_variants = [i for i, var in enumerate(n_variants) if var != 0]
    window_with_variants = [list(windows[i]) for i in index_with_variants]

    # Find windows with unique variants
    if len(window_with_variants) > 1:
      positions, windows, counts = allel.windowed_statistic(
          pos, pos, statistic=variant_positions, windows=window_with_variants
      )
      unique_var, unique_var_index = np.unique(positions, return_index=True)
      unique_windows = [list(window_with_variants[i]) for i in unique_var_index]
    else:
      unique_windows = window_with_variants
    print('Number of unique windows within region: ', len(unique_windows))
    # Count occurances of each unique allele
    values = allel.GenotypeDaskArray(variant_dataset["call_genotype"].data.astype(int))

    allele_counts, windows, counts = allel.windowed_statistic(
        pos,
        values,
        statistic=unique_allele_counts_in_window,
        windows=unique_windows,
        fill=[0, None, None],
    )
    n_variants, windows = allel.windowed_count(
        pos, windows=unique_windows
    )

    return n_variants, allele_counts, windows

In [24]:
def evaluate_marker_options(
    variant_dataset, chrom, region_df, window_length=200, step=50
):

    # Filter variants to chromosome and set index
    variant_dataset = filter_variants(variant_dataset, "variant_chrom", chrom)
    variant_dataset = variant_dataset.set_index(
        variants="variant_position", samples="sample_id"
    )

    # Find core region boundaries for chromosome
    core_region_df = region_df.loc[
        (region_df.chrom == chrom) & (region_df.name == "Core")
    ]

    biallelic_counts = []
    unique_allele_counts = []
    unique_alleles_with_missing = []
    unique_alleles_with_het = []
    window_start = []
    window_end = []
    variant_counts = []

    # For each region
    for index, row in core_region_df.iterrows():
        print(f"starting sliding window for region: {row.chromStart}-{row.chromEnd}")

        # Restrict variants to region
        variant_dataset_region = variant_dataset.sel(
            variants=slice(row.chromStart, row.chromEnd)
        )

        # STATS
        n_variants, allele_counts, windows = calculate_stats(
            variant_dataset_region, window_length, step
        )

        # Concatenate results
        window_start = window_start + list(windows[:, 0])
        window_end = window_end + list(windows[:, 1])
        variant_counts = variant_counts + list(n_variants)
        unique_allele_counts = unique_allele_counts + list(
            allele_counts[:, 0]
        )
        unique_alleles_with_missing = unique_alleles_with_missing + list(
            allele_counts[:, 1]
        )
        unique_alleles_with_het = unique_alleles_with_het + list(allele_counts[:, 2])

    return (
        variant_counts,
        unique_allele_counts,
        unique_alleles_with_missing,
        unique_alleles_with_het,
        window_start,
        window_end,
    )

# Perform sliding window and entropy and heterozygosity for all chromosomes

The code below implements the functions defined above to perform the sliding window for each chromosome. It takes the results and uses them to calculate the entropy and heterozygosity using the following formulas
       
entropy = - $\sum$ (gt_freqs * $\log(gt\_freqs)$)

heterozygosity = 1 - $\sum$ gt_freqs<sup>2
    
where `gt_freqs` are the counts for each unique allele divided by the total count

In [25]:
chromosomes = np.unique(variant_dataset_filtered["variant_chrom"].data.compute())

In [ ]:
%%time
print('Chromosomes to be scanned: ', chromosomes)

for chrom in chromosomes:
    print('Chromosome:',chrom)
    # Calculate window stats
    (
        variant_counts,
        unique_allele_counts,
        unique_alleles_with_missing,
        unique_alleles_with_het,
        window_start,
        window_end,
    ) = evaluate_marker_options(variant_dataset_filtered, chrom, pv4_regions)

    # Format data
    df = pd.DataFrame(
        data={
            "window_start": window_start,
            "window_end": window_end,
            "variant_counts": variant_counts,
            "unique_allele_counts": unique_allele_counts,
            "unique_alleles_with_missing_index": unique_alleles_with_missing,
            "unique_alleles_with_het_index": unique_alleles_with_het,
        }
    )

    # Calculate entropy and hetrozygosity
    unique_allele_count = []
    unique_allele_freqs = []
    entropy = []
    het = []
    df_with_stats = df.copy()
    for index, row in df.iterrows():
        gt_counts = row.unique_allele_counts
        n_alleles = len(gt_counts)
        gt_freqs = gt_counts/sum(gt_counts)

        unique_allele_freqs.append(list(gt_freqs))
        unique_allele_count.append(n_alleles)
        entropy.append(-np.sum(gt_freqs * np.log(gt_freqs)))
        het.append(1.0 - np.sum(gt_freqs ** 2))
    df_with_stats["unique_allele_frequencies"] = unique_allele_freqs
    df_with_stats["unique_allele_count"] = unique_allele_count
    df_with_stats["entropy"] = entropy
    df_with_stats["het"] = het

    # Output to csv
    df_with_stats.to_csv(f"sliding_window_results/{chrom}_windowed_heterozygosity.csv")

Chromosomes to be scanned:  ['PvP01_01_v1' 'PvP01_02_v1' 'PvP01_03_v1' 'PvP01_04_v1' 'PvP01_05_v1'
 'PvP01_06_v1' 'PvP01_07_v1' 'PvP01_08_v1' 'PvP01_09_v1' 'PvP01_10_v1'
 'PvP01_11_v1' 'PvP01_12_v1' 'PvP01_13_v1' 'PvP01_14_v1']
Chromosome: PvP01_01_v1
starting sliding window for region: 116542-677963
Number of unique windows within region:  401
